<a href="https://colab.research.google.com/github/alvarorl231/ResearchMethods/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scrapping

## Imports, etc.

In [1]:
!pip install BeautifulSoup4
!pip install requests

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Declaración de Variables

In [3]:
# Lista que contendrá toda la información
frases = []

# Lista que almacenara el contenido sensible
sensitive=[]

# Lista que almacenara el contenido NO sensible
nonsensitive=[]

# Keywords
keywords = ['ametralladora', 'arma', 'ataque', 'batallón', 'bomba', 
             'capitán', 'comandante', 'compañía', 'disparo', 'herido',
             'muerto', 'munición', 'oficial', 'tiro', 'tiroteo']


## Request y Extracción de información 

### Periódico: [ElDiario](https://www.eldiario.es/internacional)

In [4]:
eldiario_url = "https://www.eldiario.es/internacional"

eldiario_result = requests.get(eldiario_url)

doc = BeautifulSoup(eldiario_result.text, "html.parser")

eldiario_html_tags = doc.find_all("h2")


for text in eldiario_html_tags:
    text = str(text.string)
    frases.append(text)

### Periódico: [El País](https://elpais.com/ultimas-noticias/)

In [5]:
elpais_url = "https://elpais.com/ultimas-noticias/"

elpais_result = requests.get(elpais_url)

doc = BeautifulSoup(elpais_result.text, "html.parser")

elpais_html_tags = doc.find_all("h2")

for text in elpais_html_tags:
    text = str(text.string)
    frases.append(text)

### Periódico: [El Mundo](https://www.elmundo.es/ultimas-noticias.html)

In [6]:
elmundo_url = "https://www.elmundo.es/ultimas-noticias.html"

elmundo_result = requests.get(elmundo_url)

doc = BeautifulSoup(elmundo_result.text, "html.parser")

elmundo_html_tags = doc.find_all("h2")

for text in elmundo_html_tags:
    text = str(text.string)
    frases.append(text)

### Periódico: [La Razón](https://www.larazon.es/internacional/)

Este periódico tiene la propiedad de que muestran las noticias en webs con urls del estilo:
```
https://www.larazon.es/internacional/
https://www.larazon.es/internacional/1/
https://www.larazon.es/internacional/2/
.
.
.
.
https://www.larazon.es/internacional/n/
```
Esto facilita mucho el scrapping, ya que al conseguir extraer los datos de una página, para las n siguientes solo debes aplicar un bucle.

De ese modo, la función `get_larazon_tags` nos ayudara a agilizar el proceso y a obtener una mayor limpieza en el código

In [7]:
def get_larazon_tags (url):

 result = requests.get(url)

 doc = BeautifulSoup(result.text, "html.parser")

 html_tags = doc.find_all("span")
 
 return html_tags

Este bucle tiene una cierta característica por esto: 
```python
for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
```
En concreto el casting `text = str(text.string)` y la condición `if len(text) > 15:`, utilizados para obtener información útil y deshacernos de cadenas sin utilidad como: `None, Cerrar sesión, Submenú, etc..` 

In [8]:
for i in range(1,2382):
  if i == 1:
   larazon_url = "https://www.larazon.es/internacional/"
   tags = get_larazon_tags(larazon_url)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)
  else:
   new_url = f"https://www.larazon.es/internacional/{i}/"
   tags = get_larazon_tags(new_url)
   for text in tags:
    text = str(text.string)
    if len(text) > 15:
     frases.append(text)

Una vez realizado esto, con este simple `for`, almacenamos la información obtenidos

## Selección de palabras sensibles

Tras esto limpiamos la lista de frases obtenida anteriormente y la empleamos para buscar el contenido sensible

In [24]:
# Ahora vamos eliminar duplicados con set()
frases = list(set(frases))

# Por cada keyword busca en todos los elementos de la lista frases, si se da esta condición, se almacena.
for i in keywords:
  for j in range(len(frases)):
    if i in frases[j]:
      sensitive.append(frases[j])

sensitive = list(set(sensitive))

In [25]:
nonsensitive = [i for i in frases if i not in sensitive]
nonsensitive = list(set(nonsensitive))

## Almacenar los datos en archivos `.csv`

In [26]:
# Todos los títulos de las noticias
df = pd.DataFrame(frases)
df.to_csv("frases.csv", index=False)

# Todos los títulos SENSIBLES
df = pd.DataFrame(sensitive)
df.to_csv("sensitive_frases.csv", index=False)

# Todos los títulos NO SENSIBLES

df = pd.DataFrame(nonsensitive)
df.to_csv("nonsensitive_frases.csv", index=False)